# Jupyterclass Helper
---
These files are used to configure and organize the website's contents.

In [15]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
#Set the book.
book='quant'
syllabus_message = """You should always look to this website for the most recent updates.\n\n
An initial version of the syllabus can be found below. \n\n"""
bc_class_header='\nThe following materials will be covered during the class session.\n'
in_class_header='\nPlease review following materials before the class session.\n'
assignments_header='\nAll assignments should be submitted to the LMS at 11:59 PM on the date indicated.\n'
schedule_header="\nPlease check here for the latest course schedule of activities.\n"

In [17]:
# Always run this before any of the following cells
import pandas as pd
import numpy as np
import csv
import logging
import subprocess
import yaml
import builder as bd
from pathlib import Path
base_path=Path('..')
book_path= base_path / book
content_path = book_path / 'content'

In [18]:
# These load configuration from the excel files 
config = bd.load_yaml_file(book_path / '_config.yml') # Load the file.
toc = bd.load_yaml_file(book_path / '_toc.yml') # Load the file.
config_xl= pd.read_excel('../book.xlsx', sheet_name = '_config_yml', header=None, index_col=None)
schedule= pd.read_excel('../book.xlsx', sheet_name = 'Schedule',  index_col=None)
content={}
content['Before Class']= pd.read_excel('../book.xlsx', sheet_name = 'Before',  index_col=None)
content['In Class']= pd.read_excel('../book.xlsx', sheet_name = 'During',  index_col=None)
content['Assignment']= pd.read_excel('../book.xlsx', sheet_name = 'Assignments',  index_col=None)


In [19]:
#Create the syllabus link.
#The second value of the index postion of the syllabus on the before class content.
bd.create_syllabus(content['Before Class'],0,syllabus_message,content_path / 'syllabus.md', config['repository']['url'])

files/syllabus.pdf
Outputting  ../quant/content/syllabus.md


'Syllabus\n============================\n\nYou should always look to this website for the most recent updates.\n\n\nAn initial version of the syllabus can be found below. \n\n\n[Syllabus](https://github.com/rpi-techfundamentals/ms-website-fall-2020/raw/master/files/syllabus.pdf)\n'

In [6]:
#Fix in case individual tries to publish where session is NA. This isn't allowed. 
schedule.loc[schedule['Session'].isna(),'Publish']=0. 

In [7]:
#Generate Links from the schedule to the sessions and within the other tables. 
schedule.loc[schedule['Publish']==1,'Location']=schedule.loc[schedule['Publish']==1,'Session'].apply(lambda x: '../sessions/session'+str(int(x)))
schedule.loc[schedule['Publish']==1,'Type']='Markdown'
schedule=bd.link_generator(schedule, 'Summary',config['repository']['url'],'Link')
content['Assignment']=bd.link_generator(content['Assignment'], 'Assignment',config['repository']['url'],'Starter')
content['Before Class']=bd.link_generator(content['Before Class'], 'Content',config['repository']['url'],'Link')
content['In Class']=bd.link_generator(content['In Class'], 'Content',config['repository']['url'],'Link')

In [8]:
#Get the in class activities and prepare and output a markdown file. 
schedule_ic=schedule.merge(content['In Class'], left_on='Session', right_on='Session', how='left')
schedule_ic= schedule_ic.loc[schedule_ic['Content'].notnull(),['Week', 'Session', 'Date', 'Content']]
schedule_ic=bd.pandas_to_md(schedule_ic, content_path / 'in_class.md', 'In Class', \
        include = ['Week', 'Session', 'Date', 'Content'], header=in_class_header)

Converting datetime to 
Outputting file: ../quant/content/in_class.md


In [9]:
#Get the before class activities and prepare and output a markdown file. 
schedule_bc=schedule.merge(content['Before Class'], left_on='Session', right_on='Session', how='left')
schedule_bc= schedule_bc.loc[schedule_bc['Content'].notnull(),['Week', 'Session', 'Date', 'Content']]
schedule_bc=bd.pandas_to_md(schedule_bc, content_path / 'preparation.md', 'Before Class', \
                             include = ['Week', 'Session', 'Date', 'Content'], header=bc_class_header)
schedule=schedule.merge(content['Assignment'], left_on='Session', right_on='Session', how='left')



Converting datetime to 
Outputting file: ../quant/content/preparation.md


In [10]:
#Get the assignments and prepare and output a markdown file. 
assignments_new = schedule.loc[schedule['Assignment'].notnull(),['Week', 'Session', 'Date', 'Assignment', 'Due']]
assignments_new=bd.pandas_to_md(assignments_new, content_path / 'assignments.md', 'Assignments', \
                             include = ['Week', 'Session', 'Date', 'Assignment', 'Due'],header=assignments_header)

Converting datetime to 
Converting datetime to 
Outputting file: ../quant/content/assignments.md


In [11]:
#Output the schedule to markdown.
schedule=bd.pandas_to_md(schedule, content_path / 'schedule.md', 'Schedule', \
                             include = ['Week', 'Session', 'Date', 'Day', 'Topic', 'Summary', 'Assignment', 'Due'],header=schedule_header)

Converting datetime to 
Converting datetime to 
Outputting file: ../quant/content/schedule.md


In [12]:
#Generate Session Files
toc=bd.generate_sessions(config, toc, 2, schedule, book_path / 'sessions',content, ['Before Class', 'In Class', 'Assignment'])

Outputting  session1.md
Outputting  session2.md
Outputting  session3.md
Outputting  session4.md


In [13]:
#Update the sessions to the yaml file.  Other updates to notebooks need to be done manually.
bd.update_yaml_file(book_path / '_toc.yml', toc)

Updating the file: ../quant/_toc.yml


In [14]:
#TBD Make it so that notebooks will show up in _toc.yml. 